### Sec. -1 Initiation


In [1]:
import os
import pandas as pd
from utils.config_helper import update_nested_toml, load_config
os.makedirs("logs", exist_ok=True)


In [92]:
# TYPE = input("Enter the type of the config file: ")
TYPE = "prostate"
CONFIG_PATH = f"../config/{TYPE}.toml"
config = load_config(CONFIG_PATH)

In [3]:
train_df_file = config["init"]["hyper"]["train_df_file"]
dmp_files = config["init"]["hyper"]["dmp_files"]
majority_out_path = config["init"]["hyper"]["majority_out_path"]

### Sec. 0 process norm examples


#### 1. Split Dataset Example


In [4]:
import json
from utils.process_norm import *

In [5]:
# read the json file
# cancer_type = input("Enter the cancer type: ")
cancer_type = "prostate"
with open(f"../config/{cancer_type}.json") as f:
    J = json.load(f)

In [6]:
# data_source = input("Enter the raw all_beta_normalized file: ")
data_source = "GDC_prostate_tissue"
df = pd.read_csv(J[data_source]['file'])


In [ ]:
ID = list(df.iloc[:, 0]) + ["label"]
sample_count = 2
normal_count = 50
tumor_count = 502
X = df.iloc[:, 1::sample_count].T
print(X.shape[0],normal_count,tumor_count)

In [8]:
dfo = organize_dataset(df, J[data_source]["normal"], J[data_source]["tumor"], J[data_source]["sample_count"])
complement_df, ratio_df = split_dataset(dfo, J[data_source]["split_test"], J[data_source]["random_state"])
complement_df.to_csv("all_beta_normalized_complement.csv", index=False)
ratio_df.to_csv("all_beta_normalized_ratio.csv", index=False)

INFO complement_df feature: 373340
INFO complement_df sample (normal, tumor): (40, 401)
INFO ratio_df feature: 373340
INFO ratio_df sample (normal, tumor): (10, 101)


In [11]:
ratio_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,101,102,103,104,105,106,107,108,109,110
0,cg00000957,0.851698,0.875340,0.873949,0.845864,0.845313,0.855674,0.852747,0.856824,0.892437,...,0.870900,0.878546,0.815294,0.832045,0.873123,0.867799,0.893290,0.874944,0.834560,0.849045
1,cg00001349,0.831998,0.818831,0.777109,0.773967,0.910121,0.827794,0.851243,0.779308,0.902253,...,0.842825,0.891249,0.671433,0.855471,0.771970,0.838953,0.860958,0.932942,0.883742,0.816816
2,cg00001583,0.727021,0.260109,0.258795,0.144976,0.059293,0.216979,0.070264,0.094708,0.383988,...,0.597724,0.337256,0.311713,0.558074,0.572805,0.611514,0.455211,0.660695,0.753027,0.671198
3,cg00002028,0.020635,0.019511,0.026367,0.027506,0.033701,0.040187,0.028661,0.020410,0.008625,...,0.023539,0.057423,0.046084,0.028183,0.032069,0.033905,0.032445,0.025413,0.040462,0.045968
4,cg00002719,0.764914,0.070651,0.027768,0.058274,0.053425,0.101438,0.128026,0.048165,0.522899,...,0.493793,0.702902,0.109746,0.408147,0.415052,0.406784,0.736912,0.671877,0.307906,0.693976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.955041,0.963189,0.962567,0.948103,0.959346,0.966859,0.949644,0.946614,0.970719,...,0.920759,0.976222,0.946938,0.936118,0.941034,0.956478,0.947787,0.980815,0.864471,0.890726
373337,cg27657537,0.589816,0.215809,0.296199,0.204987,0.132361,0.317090,0.106829,0.127561,0.576276,...,0.445997,0.622140,0.293959,0.389596,0.530595,0.532515,0.638457,0.685113,0.420863,0.478446
373338,cg27662611,0.023314,0.044780,0.038949,0.047586,0.045769,0.074387,0.040437,0.028859,0.047046,...,0.030432,0.066494,0.059676,0.031325,0.030693,0.032223,0.045322,0.094985,0.032344,0.046670
373339,cg27665648,0.910125,0.879289,0.894013,0.908067,0.935242,0.922943,0.860689,0.870091,0.920194,...,0.865312,0.903358,0.832897,0.915230,0.915200,0.898776,0.858458,0.908942,0.725667,0.858434


#### 2. Merge Dataset Example


In [3]:
df0 = pd.read_csv(f"../stomach/champ_result/gdc_stomach_GSE99553/all_beta_normalized_0.csv")
df1 = pd.read_csv(f"../stomach/champ_result/gdc_stomach_GSE99553/all_beta_normalized_GSE99553.csv")

In [4]:
df0 = organize_dataset(df0, 2, 395, 2)
df1 = organize_dataset(df1, 84, 0, 1)

In [ ]:
merged_df = merge_datasets(df0, df1)
merged_df.to_csv("merged.csv", index=False)

#### 3. Inspect NaN Example


In [9]:
# Example usage:
df = pd.DataFrame({
    'ID': ['A1', 'A2', 'A3', 'A4', 'label'],
    'Col1': [1, 2, None, 4, 1],
    'Col2': [5, None, 7, 8, 1],
    'Col3': [9, 10, 11, 12, 0]
})

# Column-wise check
print(inspect_nan(df, mode="column"))

# Row-wise check
print(inspect_nan(df, mode="row"))

INFO Columns with NaNs: {'Col1': np.float64(1.0), 'Col2': np.float64(1.0)}
      ID  Col1  Col2  Col3
0     A1   1.0   5.0     9
1     A2   2.0   NaN    10
2     A3   NaN   7.0    11
3     A4   4.0   8.0    12
4  label   1.0   1.0     0
INFO Rows with NaNs: ['A2', 'A3']
      ID  Col1  Col2  Col3
0     A1   1.0   5.0     9
1     A2   2.0   NaN    10
2     A3   NaN   7.0    11
3     A4   4.0   8.0    12
4  label   1.0   1.0     0


### Sec. 1 Delta Beta Calculation


In [10]:
os.makedirs(f"{majority_out_path}/section_1", exist_ok=True)

In [12]:
train_df = pd.read_csv(train_df_file)

In [13]:
# remove outlier in terms of every column
def IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + IQR * 1.5
    lower_fence = Q1 - IQR * 1.5
    return upper_fence, lower_fence


def no_outlier(df):
    upper_fence, lower_fence = IQR(df)
    ddf = df[(df > lower_fence) & (df < upper_fence)]
    return ddf

In [14]:
# get normal count by the count of 0 in the last row
normal_count = int((train_df.iloc[-1, 1:] == 0).sum())
all_beta_normalized_normal = train_df.iloc[:-1, 1 : normal_count + 1 :].T

all_beta_normalized_tumor = train_df.iloc[:-1, normal_count + 1 : :].T

In [15]:
all_beta_normalized_normal = no_outlier(all_beta_normalized_normal)
all_beta_normalized_tumor = no_outlier(all_beta_normalized_tumor)

In [16]:
train_normal_avg = all_beta_normalized_normal.mean(skipna=True, axis=0)

In [17]:
all_beta_normalized_tumor = all_beta_normalized_tumor.subtract(
    train_normal_avg, axis=1
)

In [18]:
all_beta_normalized_tumor = no_outlier(all_beta_normalized_tumor)

In [19]:
train_tumor_mean = all_beta_normalized_tumor.mean(skipna=True, axis=0)

In [20]:
delta_beta = pd.merge(
    train_df.iloc[:-1, :1],
    pd.DataFrame(train_tumor_mean, columns=["dbeta"]),
    left_index=True,
    right_index=True,
)


In [21]:
# record the list of feature with dbeta being NaN
update_nested_toml("preprocess.dbeta", "delta_beta_avg_feature_num", delta_beta.shape[0])
update_nested_toml(
    "preprocess.dbeta",
    "NaN_dbeta_feature",
    delta_beta.loc[pd.isna(delta_beta["dbeta"]), "Unnamed: 0"].tolist(),
)
delta_beta = delta_beta.dropna(axis=0)
update_nested_toml("preprocess.dbeta", "delta_beta_avg_feature_num_remove_NaN", delta_beta.shape[0])

In [22]:
dmp = pd.read_csv("../prostate/champ_result/GDC_prostate_tissue/DMP_result_TN.csv")
dmp = dmp[["Unnamed: 0", "gene", "feature"]]

In [23]:
update_nested_toml("preprocess.dbeta", "dmp_before_dropna_shape_feature", dmp.shape[0])

dmp = dmp.dropna(axis=0)

update_nested_toml("preprocess.dbeta", "dmp_after_dropna_shape_feature", dmp.shape[0])

In [24]:
result = pd.merge(delta_beta, dmp, on="Unnamed: 0", how="left")
update_nested_toml(
    "preprocess.dbeta", "delta_beta_avg_feature_num_remove_NaN_join_dmp", result.shape[0]
)

In [25]:
def find_max_dBeta_grouped(group):
    idx_max = group["dbeta"].abs().idxmax()
    return group.loc[idx_max]


dbeta_info = result.groupby("gene", as_index=False).apply(
    find_max_dBeta_grouped, include_groups=False
)

In [26]:
dbeta_info.columns = ["gene", "ID", "dbeta", "feature"]
dbeta_info = dbeta_info[["ID", "gene", "dbeta", "feature"]]

In [27]:
dbeta_info

,ID,gene,dbeta,feature
0,cg03630821,A1BG,0.159133,Body
1,cg27394794,A1CF,-0.327635,Body
2,cg07196505,A2BP1,-0.416200,Body
3,cg00336946,A2LD1,-0.251164,TSS1500
4,cg00134295,A2M,-0.193440,TSS1500
...,...,...,...,...
18647,cg00692173,ZYG11B,0.175015,Body
18648,cg18074834,ZYX,0.042229,Body
18649,cg16463044,ZZEF1,-0.148617,Body
18650,cg22221847,ZZZ3,-0.031145,3'UTR


In [ ]:
# comorbidity = pd.read_csv(
#     "../external_result/matchgene174_single_3Y10__OR2.txt", sep="\t", header=None
# )
# dbeta = dbeta[
#     dbeta["gene"].isin(comorbidity[0])
# ]

# result_max_per_gene_single

In [28]:
dbeta_info["dbeta"] = dbeta_info["dbeta"].apply(lambda x: round(x, 6))
dbeta_info.to_csv(f"{majority_out_path}/section_1/dbeta.csv", index=False)

### Sec. 2 Filter Genes by Average Delta Beta Values


#### 2.1 Filtering TSS


In [29]:
os.makedirs(f"{majority_out_path}/section_2", exist_ok=True)

In [ ]:
# dbeta_info = pd.read_csv(f"{majority_out_path}/dbeta.csv")

In [30]:
TSS = dbeta_info[dbeta_info["feature"].str.contains("TSS")]

In [31]:
TSS.to_csv(f"{majority_out_path}/section_2/dbeta_TSS.csv", index=False)

#### 2.2 Thresholding


In [32]:
threshold = 1
dbeta_TSS_threshold = TSS[abs(TSS["dbeta"]) > threshold]
while True:
    dbeta_TSS_threshold = TSS[abs(TSS["dbeta"]) > threshold]
    count = dbeta_TSS_threshold.shape[0]
    if (
        config["preprocess"]["filtering"]["hyper"]["avg_dbeta_lower_bound"]
        <= count
        <= config["preprocess"]["filtering"]["hyper"]["avg_dbeta_upper_bound"]
    ):
        break
    threshold -= 0.01
threshold = round(threshold, 2)
update_nested_toml("preprocess.filtering", "threshold", threshold)

In [33]:
dbeta_TSS_threshold.to_csv(f"{majority_out_path}/section_2/dbeta_TSS_{threshold}.csv", index=False)

#### 2.3 Visualization


In [34]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(dbeta_TSS_threshold["dbeta"])
plt.xlabel("delta Beta value")
plt.title("Density plot of delta Beta value")
plt.savefig(f"{majority_out_path}/section_2/dbeta_TSS_{threshold}.png")
plt.close()

In [ ]:
# train_df = pd.read_csv(train_df_file)

In [35]:
train_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,431,432,433,434,435,436,437,438,439,440
0,cg00000957,0.841180,0.853063,0.860960,0.862565,0.895689,0.831980,0.863311,0.845822,0.828084,...,0.885562,0.830317,0.852356,0.849449,0.869412,0.854675,0.892581,0.880069,0.857188,0.893333
1,cg00001349,0.708645,0.720905,0.860959,0.797306,0.891208,0.670794,0.715566,0.709133,0.814481,...,0.931344,0.724758,0.853462,0.878646,0.821124,0.833564,0.583747,0.876863,0.869314,0.785916
2,cg00001583,0.113957,0.279087,0.112048,0.046604,0.556086,0.110287,0.095149,0.072687,0.115331,...,0.731181,0.298437,0.357947,0.255105,0.493925,0.341537,0.132613,0.502308,0.739241,0.228809
3,cg00002028,0.030409,0.035949,0.022765,0.027793,0.016043,0.016709,0.028100,0.017500,0.039319,...,0.039711,0.027775,0.038226,0.039430,0.018812,0.027809,0.087653,0.037435,0.040489,0.031151
4,cg00002719,0.052727,0.045736,0.026899,0.042019,0.046145,0.051344,0.032892,0.030961,0.066875,...,0.821091,0.300829,0.447518,0.165109,0.408629,0.751978,0.368727,0.507314,0.769421,0.397939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.949989,0.948849,0.963573,0.951192,0.968733,0.947092,0.927369,0.953028,0.960975,...,0.974683,0.925931,0.935257,0.954601,0.941680,0.932389,0.936980,0.913569,0.853531,0.951475
373337,cg27657537,0.080608,0.277116,0.197913,0.101441,0.315259,0.070351,0.249224,0.202402,0.183052,...,0.736022,0.481165,0.407895,0.178566,0.468870,0.740228,0.389476,0.503911,0.488822,0.575861
373338,cg27662611,0.050093,0.042346,0.071625,0.065757,0.033946,0.018048,0.065646,0.024260,0.036035,...,0.077809,0.061533,0.044317,0.074580,0.028676,0.072972,0.048652,0.052870,0.057997,0.024181
373339,cg27665648,0.869885,0.879406,0.932071,0.931462,0.932957,0.878828,0.893683,0.875100,0.911748,...,0.902220,0.893676,0.873291,0.844760,0.865644,0.873046,0.902961,0.884115,0.899215,0.914803


In [36]:
beta_df = train_df.iloc[:-1, :]
beta_df = beta_df[beta_df["Unnamed: 0"].isin(dbeta_info["ID"])]
X = beta_df.iloc[:, 1:].dropna(axis=0).T
y = train_df.iloc[-1, 1:].astype(int).to_list()

In [37]:
# DEBUG
print(f"X shape: {X.shape}")
print(f"y shape: {len(y)}")
# END

X shape: (441, 18652)
y shape: 441


In [38]:
import plotly.express as px
import pandas as pd
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

df = pd.DataFrame(
    {
        "Principal Component 1": X_pca[:, 0],
        "Principal Component 2": X_pca[:, 1],
        "Principal Component 3": X_pca[:, 2],
        "Class": y,
    }
)
fig = px.scatter_3d(
    df,
    x="Principal Component 1",
    y="Principal Component 2",
    z="Principal Component 3",
    color="Class",
    title="PCA of Dataset",
    color_continuous_scale="Viridis",
)

fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3",
    )
)

fig.write_html(f"{majority_out_path}/section_2/preprocess_filtering_pca.html")


#### remember to run clustering on the filtered genes first continue to the next step


### Sec. 3 Feature Selection with ML (SFS)

sequential forward selection


#### Remove previous results

Warning: This step is not reversible


In [6]:
import shutil
import os
if os.path.exists(f"{majority_out_path}/sfs"):
    shutil.rmtree(f"{majority_out_path}/sfs")


#### 3.1 Preparation(SFS)


In [ ]:
config = load_config(CONFIG_PATH)

train_out_path = config["feature_selection"]["hyper"]["train_out_path"]
validate_out_path = config["feature_selection"]["hyper"]["validate_out_path"]

In [ ]:
os.makedirs(f"{train_out_path}", exist_ok=True)
os.makedirs(f"{validate_out_path}", exist_ok=True)

In [12]:
config = load_config(CONFIG_PATH)
dbeta_info_file = config["feature_selection"]["hyper"]["dbeta_info_file"]
dbeta_info = pd.read_csv(f"{majority_out_path}/{dbeta_info_file}")

In [8]:
# check if logs/ folder exists
os.makedirs("logs", exist_ok=True)
from utils.train_helper import TrainHelper

In [ ]:
del th

In [14]:
# note that there is setup_dbeta in TrainHelper to further cut down the feature size
th = TrainHelper(dbeta_info)

#### 3.2 Selection(SFS)


In [10]:
train_df = pd.read_csv(f"{majority_out_path}/{majority_df_path}")
validate_df = pd.read_csv(f"{minority_out_path}/{minority_df_path}")
th.set_train_validate_df(train_df, validate_df)

In [15]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

selection_models = {
    "SVM": SVC(kernel="linear", random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(
        random_state=42,
        n_estimators=10,
    ),
    "XGBoost": XGBClassifier(
        random_state=42,
        n_estimators=10,
        ),
}

In [12]:
th.set_selection_models(selection_models)
th.set_train_validate()

In [ ]:
os.makedirs(f"{majority_out_path}/sfs", exist_ok=True)

th.select_feature_sfs(
    out_path = f"{majority_out_path}/sfs/selected_feature.txt",
    step= 4,
    n_features_to_select="cluster"
)

### Sec. 3 Feature Selection with ML (RFE)

recursive feature elimination


#### Remove previous results

Warning: This step is not reversible


In [16]:
# import shutil
# import os
# if os.path.exists(f"{majority_out_path}/section_3/rfe"):
#     shutil.rmtree(f"{majority_out_path}/section_3/rfe")
# if os.path.exists(f"{minority_out_path}/section_3/rfe"):
#     shutil.rmtree(f"{minority_out_path}/section_3/rfe")

#### 3.1 Preparation


In [66]:
config = load_config(CONFIG_PATH)

train_out_path = config["feature_selection"]["hyper"]["train_out_path"]
validate_out_path = config["feature_selection"]["hyper"]["validate_out_path"]
training_param_file = config["feature_selection"]["hyper"]["training_param_file"]

In [67]:
os.makedirs(f"{train_out_path}", exist_ok=True)
os.makedirs(f"{validate_out_path}", exist_ok=True)

In [68]:
dbeta_info_file = config["feature_selection"]["hyper"]["dbeta_info_file"]

TSS_threshold = pd.read_csv(dbeta_info_file)

In [69]:
from utils.train_helper import TrainHelper

In [70]:
# note that there is setup_dbeta in TrainHelper to further cut down the feature size
th = TrainHelper(TSS_threshold)

#### 3.2 Selection


In [71]:
from utils.process_norm import *

In [72]:
train_df = pd.read_csv(train_df_file)
validate_df_file = config["feature_selection"]["hyper"]["validate_df_file"]
validate_df = pd.read_csv(validate_df_file)

In [ ]:
# train_df = inspect_nan(train_df, mode="column", remove=True)

In [73]:
inspect_nan(train_df, mode="column")
inspect_nan(validate_df, mode="column")

INFO Columns with NaNs: {}
INFO Columns with NaNs: {}


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,101,102,103,104,105,106,107,108,109,110
0,cg00000957,0.851698,0.875340,0.873949,0.845864,0.845313,0.855674,0.852747,0.856824,0.892437,...,0.870900,0.878546,0.815294,0.832045,0.873123,0.867799,0.893290,0.874944,0.834560,0.849045
1,cg00001349,0.831998,0.818831,0.777109,0.773967,0.910121,0.827794,0.851243,0.779308,0.902253,...,0.842825,0.891249,0.671433,0.855471,0.771970,0.838953,0.860958,0.932942,0.883742,0.816816
2,cg00001583,0.727021,0.260109,0.258795,0.144976,0.059293,0.216979,0.070264,0.094708,0.383988,...,0.597724,0.337256,0.311713,0.558074,0.572805,0.611514,0.455211,0.660695,0.753027,0.671198
3,cg00002028,0.020635,0.019511,0.026367,0.027506,0.033701,0.040187,0.028661,0.020410,0.008625,...,0.023539,0.057423,0.046084,0.028183,0.032069,0.033905,0.032445,0.025413,0.040462,0.045968
4,cg00002719,0.764914,0.070651,0.027768,0.058274,0.053425,0.101438,0.128026,0.048165,0.522899,...,0.493793,0.702902,0.109746,0.408147,0.415052,0.406784,0.736912,0.671877,0.307906,0.693976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.955041,0.963189,0.962567,0.948103,0.959346,0.966859,0.949644,0.946614,0.970719,...,0.920759,0.976222,0.946938,0.936118,0.941034,0.956478,0.947787,0.980815,0.864471,0.890726
373337,cg27657537,0.589816,0.215809,0.296199,0.204987,0.132361,0.317090,0.106829,0.127561,0.576276,...,0.445997,0.622140,0.293959,0.389596,0.530595,0.532515,0.638457,0.685113,0.420863,0.478446
373338,cg27662611,0.023314,0.044780,0.038949,0.047586,0.045769,0.074387,0.040437,0.028859,0.047046,...,0.030432,0.066494,0.059676,0.031325,0.030693,0.032223,0.045322,0.094985,0.032344,0.046670
373339,cg27665648,0.910125,0.879289,0.894013,0.908067,0.935242,0.922943,0.860689,0.870091,0.920194,...,0.865312,0.903358,0.832897,0.915230,0.915200,0.898776,0.858458,0.908942,0.725667,0.858434


In [74]:
th.set_train_validate_df(train_df, validate_df)

In [76]:
from utils.train_helper import set_parameters
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import json

with open(training_param_file, "r") as f:
    training_param = json.load(f)
xgb_grid = set_parameters(XGBClassifier(random_state=42), training_param["XGBoost"])
rf_grid = set_parameters(
    RandomForestClassifier(random_state=42), training_param["RandomForest"]
)
svm_grid = set_parameters(SVC(random_state=42, probability=True), training_param["SVM"])
dt_grid = set_parameters(
    DecisionTreeClassifier(random_state=42), training_param["DecisionTree"]
)

train_models = {
    "XGBoost": xgb_grid,
    "RandomForest": rf_grid,
    "SVM": svm_grid,
    "DecisionTree": dt_grid,
}

In [77]:
selection_models = {
    "SVM": SVC(kernel="linear", random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(
        random_state=42,
        n_estimators=10,
    ),
    "XGBoost": XGBClassifier(
        random_state=42,
        n_estimators=10,
        ),
}

In [78]:
th.set_selection_models(selection_models)
th.set_grid_estimators(train_models)
th.set_train_validate()

In [79]:
th.select_feature_rfe(
    train_out_path = train_out_path,
    validate_out_path = validate_out_path,
    selected_feature_path = f"{train_out_path}/selected_feature.txt",
    feature_range = (1, 6, 1),
)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning:

invalid value encountered in cast



Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 fo

d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning:

invalid value encountered in cast



Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits


...

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning:

invalid value encountered in cast



Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning:

invalid value encountered in cast



Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 fo

### Sec. 4 Clean Selected Features


#### 4.1 Generate Feature json for SimpleModel


In [80]:
from utils.train_helper import read_selected_features, read_selected_features_json, TrainHelper
config = load_config(CONFIG_PATH)
dbeta_info_file = config["machine_learning"]["hyper"]["dbeta_info_file"]
dbeta_info = pd.read_csv(f"{majority_out_path}/{dbeta_info_file}")
th = TrainHelper(dbeta_info)

folder = "sfs"
features = read_selected_features(f"{majority_out_path}/{folder}/selected_feature.txt")
th.generate_selected_features(
    features,
    f"{majority_out_path}/{folder}/selected_features.json",
    mode="min",
    out_format="json",
)

# use this to read json
read_selected_features_json(f"{majority_out_path}/{folder}/selected_features.json")

KeyError: 'machine_learning'

#### 4.2 Gather Selected gene list from best selection model


In [81]:
rfe_train = pd.read_csv(f"{train_out_path}/rfe.csv")
rfe_validate = pd.read_csv(f"{validate_out_path}/rfe.csv")
fpr_tpr_train = pd.read_csv(f"{train_out_path}/roc_curve.csv")
fpr_tpr_validate = pd.read_csv(f"{validate_out_path}/roc_curve.csv")
rfe_j = pd.merge(rfe_train, rfe_validate, on=["selection_model", "train_model", "features"], suffixes=('_train', '_validate'))
fpr_tpr_j = pd.merge(fpr_tpr_train, fpr_tpr_validate, on=["selection_model", "train_model", "features"], suffixes=('_train', '_validate'))
J = pd.merge(rfe_j, fpr_tpr_j, on=["selection_model", "train_model", "features"])

In [82]:
import ast

J["fpr_train"] = J["fpr_train"].apply(ast.literal_eval)
J["tpr_train"] = J["tpr_train"].apply(ast.literal_eval)
J["fpr_validate"] = J["fpr_validate"].apply(ast.literal_eval)
J["tpr_validate"] = J["tpr_validate"].apply(ast.literal_eval)

In [83]:
from utils.painter import plot_roc_curve, create_performance_barchart

In [84]:
J['accuracy_diff'] = J['accuracy_train'] - J['accuracy_validate']
J['recall_diff'] = J['recall_train'] - J['recall_validate']
J['f1_score_diff'] = J['f1_score_train'] - J['f1_score_validate']
J['AUC_diff'] = J['AUC_train'] - J['AUC_validate']
J['MCC_diff'] = J['MCC_train'] - J['MCC_validate']
J['fbeta2_score_diff'] = J['fbeta2_score_train'] - J['fbeta2_score_validate']

In [85]:
# tweakable width and height
plot_roc_curve(
    J, 
    "ROC Curves on Training Set", 
    f"{train_out_path}/roc_curve.html",
    x_column = "fpr_train",
    y_column = "tpr_train",
    trace_name = ["selection_model", "train_model", "features"],
)
# tweakable width and height
plot_roc_curve(
    J, 
    "ROC Curves on Testing Set", 
    f"{validate_out_path}/roc_curve.html",
    x_column = "fpr_validate",
    y_column = "tpr_validate",
    trace_name = ["selection_model", "train_model", "features"],
)

ROC curve saved to ../prostate/result/GDC_prostate_tissue/split80/section_3/rfe/roc_curve.html
ROC curve saved to ../prostate/result/GDC_prostate_tissue/split20/section_3/rfe/roc_curve.html


In [86]:
# plot difference
performance_metrics = ['accuracy_diff', 'recall_diff', 'f1_score_diff', 'AUC_diff', 'MCC_diff', 'fbeta2_score_diff']
ground_by_train_model = J.groupby('train_model')[performance_metrics].mean()
ground_by_train_model['train_model'] = ground_by_train_model.index
ground_by_train_model.to_csv(f"{validate_out_path}/performance_diff_grouped_by_train_model.csv", index=False)
color_mapping = {
    "accuracy_diff": "blue",
    "recall_diff": "red",
    "f1_score_diff": "green",
    "AUC_diff": "purple",
    "MCC_diff": "orange",
    "fbeta2_score_diff": "brown",
}
create_performance_barchart(
    df=ground_by_train_model,
    color_mapping=color_mapping,
    metric="train_model",
    out_path=f"{validate_out_path}/performance_diff_grouped_by_train_model.html",
    title="Grouped Performance Difference between Training and Testing Set",
    x_axis_label="Performance Difference (Training - Testing)",
    y_axis_label="Train Model",
    orientation="h",
)

Performance difference saved to ../prostate/result/GDC_prostate_tissue/split20/section_3/rfe/performance_diff_grouped_by_train_model.html


In [87]:
J = J[["selection_model", "train_model", "features", "accuracy_validate", "recall_validate", "f1_score_validate", "AUC_validate", "MCC_validate", "fbeta2_score_validate"]]

In [88]:
# group by train_model, for each train_model, calculate the mean of each performance metric
performance_metrics = ['accuracy_validate', 'recall_validate',
                       'f1_score_validate', 'AUC_validate', 'MCC_validate']
ground_by_train_model = J.groupby('train_model')[performance_metrics].mean()
ground_by_train_model['train_model'] = ground_by_train_model.index
ground_by_train_model.to_csv(
    f"{validate_out_path}/performance_metrics_grouped_by_train_model.csv", index=False)
color_mapping = {
    "accuracy_validate": "blue",
    "recall_validate": "red",
    "f1_score_validate": "green",
    "AUC_validate": "purple",
    "MCC_validate": "orange",
}
create_performance_barchart(
    df=ground_by_train_model,
    color_mapping=color_mapping,
    metric="train_model",
    out_path=f"{validate_out_path}/performance_metrics_grouped_by_train_model.html",
    title="Grouped Performance Metrics by Train Model",
    x_axis_label="Performance",
    y_axis_label="Train Model",
    orientation="h",
)
best_train_model = ground_by_train_model['MCC_validate'].idxmax()
print(f"Best train model: {best_train_model}")
ground_by_feature = J[J['train_model'] == best_train_model].groupby('features')[
    performance_metrics].mean()
ground_by_feature['features'] = ground_by_feature.index
ground_by_feature.to_csv(
    f"{validate_out_path}/performance_metrics_grouped_by_feature.csv", index=False)
create_performance_barchart(
    df=ground_by_feature,
    color_mapping=color_mapping,
    metric="features",
    out_path=f"{validate_out_path}/performance_metrics_grouped_by_feature.html",
    title="Grouped Performance Metrics by Feature",
    x_axis_label="Performance",
    y_axis_label="Feature",
    orientation="h",
)
best_num_of_feature = ground_by_feature['MCC_validate'].idxmax()
print(f"Best number of feature: {best_num_of_feature}")
best_performance_records = J[(J['train_model'] == best_train_model) & (
    J['features'] == best_num_of_feature)]
best_performance_records.to_csv(
    f"{validate_out_path}/best_performance_records.csv", index=False)

Performance difference saved to ../prostate/result/GDC_prostate_tissue/split20/section_3/rfe/performance_metrics_grouped_by_train_model.html
Best train model: SVM
Performance difference saved to ../prostate/result/GDC_prostate_tissue/split20/section_3/rfe/performance_metrics_grouped_by_feature.html
Best number of feature: 2


In [89]:
from utils.train_helper import read_selected_features, read_selected_features_json, TrainHelper
config = load_config(CONFIG_PATH)
dbeta_info_file = config["feature_selection"]["hyper"]["dbeta_info_file"]
dbeta_info = pd.read_csv(dbeta_info_file)
th = TrainHelper(dbeta_info)

features = read_selected_features(f"{train_out_path}/selected_feature.txt")

th.generate_selected_features(
    features,
    f"{validate_out_path}/selected_features.json",
    mode=int(best_num_of_feature),
    out_format="json",
)

# use this to read json
read_selected_features_json(f"{validate_out_path}/selected_features.json")

defaultdict(list,
            {'best': ['RARRES1',
              'GCM2',
              'CDX2',
              'HAAO',
              'RNF152',
              'ITIH5',
              'OR4K2']})

### Sec. 5 Clustering Visualization


#### 1. load data

remember to calculate distance matrix first


In [39]:
import pandas as pd
import numpy as np
from utils.clustering_helper import hierarchical_clustering, check_distance_matrix

In [45]:
result_prefix = config["clustering_visual"]["hyper"]["result_prefix"]
dbeta_file = config["clustering_visual"]["hyper"]["dbeta_file"]
bp_file = config["clustering_visual"]["hyper"]["bp_file"]
cc_file = config["clustering_visual"]["hyper"]["cc_file"]
mf_file = config["clustering_visual"]["hyper"]["mf_file"]
terms_count_file = config["clustering_visual"]["hyper"]["terms_count_file"]
result_out_path = config["clustering_visual"]["hyper"]["result_out_path"]

In [46]:
os.makedirs(result_out_path, exist_ok=True)

In [47]:
gene_set = pd.read_csv(dbeta_file, index_col=0)
distance_matrix_bp = pd.read_csv(bp_file, index_col=0)
distance_matrix_cc = pd.read_csv(cc_file, index_col=0)
distance_matrix_mf = pd.read_csv(mf_file, index_col=0)
terms_count = pd.read_csv(terms_count_file, index_col=0)

In [48]:
# replace NaN with 0
distance_matrix_bp = distance_matrix_bp.fillna(0)
distance_matrix_cc = distance_matrix_cc.fillna(0)
distance_matrix_mf = distance_matrix_mf.fillna(0)

In [49]:
# reindex distance matrix
index_bp = distance_matrix_bp.index
index_cc = distance_matrix_cc.index
index_mf = distance_matrix_mf.index
index = index_bp.union(index_cc).union(index_mf)
distance_matrix_bp_ = distance_matrix_bp.reindex(index=index, columns=index, fill_value=0)
distance_matrix_cc_ = distance_matrix_cc.reindex(index=index, columns=index, fill_value=0)
distance_matrix_mf_ = distance_matrix_mf.reindex(index=index, columns=index, fill_value=0)

In [50]:
# make a array of distance matrix for each ontology
distance_matrix = []

distance_matrix.append(distance_matrix_bp_)
distance_matrix.append(distance_matrix_cc_)
distance_matrix.append(distance_matrix_mf_)

#### 2. Weighted Sum


In [51]:
weight = [count for count in terms_count["count"]]
weight = weight / np.sum(weight)
masks = np.array([~np.isnan(distance_matrix[i].values) for i in range(3)])

valid_weights = np.array([weight[i] for i in range(3)])[:, None, None] * masks

weight_sums = valid_weights.sum(axis=0)

normalized_weights = np.divide(valid_weights, weight_sums, where=weight_sums != 0)
weighted_sum = sum(
    np.nan_to_num(distance_matrix[i].values) * normalized_weights[i] for i in range(3)
)


weighted_sum_dataframe = pd.DataFrame(weighted_sum, index=index, columns=index)

weighted_sum_dataframe.head()

,ACE,ACOT11,ACSF2,ADAMTS12,ADCY1,ADCY4,AFAP1L1,ALDH1L1,ALOX12,AMIGO2,...,VWC2,VWCE,WDR8,WFDC5,WIPF1,ZFP42,ZNF48,ZNF577,ZNF655,ZP4
ACE,0.000000,0.727107,0.806492,0.606306,0.671185,0.746510,0.224928,0.817836,0.708699,0.628079,...,0.761466,0.572512,0.705536,0.770838,0.754517,0.740835,0.797094,0.665282,0.798069,0.764372
ACOT11,0.727107,0.000000,0.465064,0.735628,0.665900,0.543755,0.227327,0.715346,0.543340,0.731698,...,0.754244,0.717917,0.747774,0.784934,0.745009,0.793883,0.795212,0.774689,0.768241,0.798035
ACSF2,0.806492,0.465064,0.000000,0.804959,0.749472,0.681629,0.200264,0.724746,0.522442,0.644839,...,0.816990,0.813980,0.644227,0.863555,0.740437,0.787791,0.738410,0.773677,0.746317,0.818444
ADAMTS12,0.606306,0.735628,0.804959,0.000000,0.674923,0.651679,0.149411,0.827005,0.700046,0.566630,...,0.598605,0.639918,0.616679,0.731404,0.668338,0.705758,0.754840,0.735780,0.742766,0.679116
ADCY1,0.671185,0.665900,0.749472,0.674923,0.000000,0.401306,0.199058,0.818644,0.747325,0.664413,...,0.720720,0.689113,0.682283,0.780435,0.739438,0.718417,0.713355,0.364442,0.727427,0.760993


In [52]:
cluster_result_weighted = hierarchical_clustering(
    weighted_sum_dataframe,
    range_min=2,
    range_max=4,
    cluster_number=3,
    out_path=f"{result_out_path}/hierarchical_clustering_weighted_sum.png",
)

chosen number of clusters: 3


In [53]:
cluster_result_weighted.head()

,gene,cluster
0,ACE,3
1,ACOT11,3
2,ACSF2,3
3,ADAMTS12,3
4,ADCY1,3


#### 3. Simple average


In [54]:
weight = [1, 1, 1]
masks = np.array([~np.isnan(distance_matrix[i].values) for i in range(3)])
valid_weights = np.array([weight[i] for i in range(3)])[:, None, None] * masks
weight_sums = valid_weights.sum(axis=0)
normalized_weights = np.divide(valid_weights, weight_sums, where=weight_sums != 0)
weighted_sum = sum(
    np.nan_to_num(distance_matrix[i].values) * normalized_weights[i] for i in range(3)
)
simple_sum_dataframe = pd.DataFrame(weighted_sum, index=index, columns=index)
simple_sum_dataframe.head()

,ACE,ACOT11,ACSF2,ADAMTS12,ADCY1,ADCY4,AFAP1L1,ALDH1L1,ALOX12,AMIGO2,...,VWC2,VWCE,WDR8,WFDC5,WIPF1,ZFP42,ZNF48,ZNF577,ZNF655,ZP4
ACE,0.000000,0.653000,0.701000,0.529000,0.587333,0.618000,0.418000,0.746667,0.589000,0.491333,...,0.666667,0.541667,0.583333,0.535000,0.679000,0.690000,0.663333,0.625333,0.697000,0.631000
ACOT11,0.653000,0.000000,0.394000,0.693667,0.634667,0.572333,0.397000,0.577000,0.495667,0.639000,...,0.698667,0.604667,0.640667,0.672000,0.653333,0.703667,0.692333,0.700000,0.635333,0.741333
ACSF2,0.701000,0.394000,0.000000,0.766667,0.650000,0.599000,0.394667,0.732333,0.567000,0.466333,...,0.759667,0.687667,0.487667,0.803667,0.646667,0.697333,0.638000,0.698667,0.617667,0.758667
ADAMTS12,0.529000,0.693667,0.766667,0.000000,0.614667,0.563667,0.316000,0.769667,0.620667,0.463667,...,0.491667,0.464667,0.508333,0.529333,0.613000,0.637333,0.665333,0.693333,0.640333,0.481000
ADCY1,0.587333,0.634667,0.650000,0.614667,0.000000,0.345333,0.371000,0.736333,0.596667,0.540667,...,0.612333,0.594000,0.562333,0.647667,0.664667,0.654333,0.641333,0.487667,0.639000,0.654667


In [55]:
cluster_result_simple = hierarchical_clustering(
    simple_sum_dataframe,
    range_min=2,
    range_max=4,
    cluster_number=3,
    out_path=f"{result_out_path}/hierarchical_clustering_simple_sum.png",
)

chosen number of clusters: 3


In [56]:
cluster_result_simple.head()

,gene,cluster
0,ACE,3
1,ACOT11,3
2,ACSF2,3
3,ADAMTS12,3
4,ADCY1,2


#### 4. Consensus clustering


In [57]:
cluster_bp = hierarchical_clustering(
    distance_matrix_bp, out_path=f"{result_out_path}/hierarchical_clustering_bp.png"
)
cluster_cc = hierarchical_clustering(
    distance_matrix_cc, out_path=f"{result_out_path}/hierarchical_clustering_cc.png"
)
cluster_mf = hierarchical_clustering(
    distance_matrix_mf, out_path=f"{result_out_path}/hierarchical_clustering_mf.png"
)

Best number of clusters: 31
Best number of clusters: 3
Best number of clusters: 2


In [58]:
cluster_bp.columns = ["gene", "cluster_bp"]
cluster_cc.columns = ["gene", "cluster_cc"]
cluster_mf.columns = ["gene", "cluster_mf"]
cluster_bp_cc = pd.merge(cluster_bp, cluster_cc, on="gene", how="outer")
cluster_go = pd.merge(cluster_bp_cc, cluster_mf, on="gene", how="outer")
cluster_go = cluster_go.fillna(-1)
print(cluster_go.shape)
cluster_go.head()

(278, 4)


,gene,cluster_bp,cluster_cc,cluster_mf
0,ACE,9.0,3.0,2.0
1,ACOT11,18.0,1.0,2.0
2,ACSF2,18.0,1.0,2.0
3,ADAMTS12,9.0,2.0,2.0
4,ADCY1,29.0,3.0,2.0


In [59]:
num_genes = cluster_go.shape[0]
consensus_matrix = np.zeros((num_genes, num_genes))
for i in range(num_genes):
    for j in range(i, num_genes):
        if cluster_go.iloc[i]["cluster_bp"] == cluster_go.iloc[j]["cluster_bp"]:
            consensus_matrix[i][j] += 1

        if cluster_go.iloc[i]["cluster_cc"] == cluster_go.iloc[j]["cluster_cc"]:
            consensus_matrix[i][j] += 1

        if cluster_go.iloc[i]["cluster_mf"] == cluster_go.iloc[j]["cluster_mf"]:
            consensus_matrix[i][j] += 1

consensus_matrix = pd.DataFrame(
    consensus_matrix, index=cluster_go["gene"], columns=cluster_go["gene"]
)
consensus_matrix += consensus_matrix.T
distance_matrix_consensus = 1 - consensus_matrix / 3
np.fill_diagonal(distance_matrix_consensus.values, 0)
distance_matrix_consensus.head()

gene,ACE,ACOT11,ACSF2,ADAMTS12,ADCY1,ADCY4,AFAP1L1,ALDH1L1,ALOX12,AMIGO2,...,VWC2,VWCE,WDR8,WFDC5,WIPF1,ZFP42,ZNF48,ZNF577,ZNF655,ZP4
gene,,,,,,,,,,,,,,,,,,,,,
ACE,0.000000,0.666667,0.666667,0.333333,0.333333,0.333333,0.666667,0.666667,0.333333,0.333333,...,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667
ACOT11,0.666667,0.000000,0.000000,0.666667,0.666667,0.666667,0.333333,0.333333,0.333333,0.666667,...,0.666667,0.666667,0.333333,0.666667,0.333333,0.333333,0.333333,0.333333,0.333333,0.666667
ACSF2,0.666667,0.000000,0.000000,0.666667,0.666667,0.666667,0.333333,0.333333,0.333333,0.666667,...,0.666667,0.666667,0.333333,0.666667,0.333333,0.333333,0.333333,0.333333,0.333333,0.666667
ADAMTS12,0.333333,0.666667,0.666667,0.000000,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,...,0.333333,0.333333,0.666667,0.333333,0.666667,0.666667,0.666667,0.666667,0.666667,0.333333
ADCY1,0.333333,0.666667,0.666667,0.666667,0.000000,0.333333,0.666667,0.666667,0.333333,0.333333,...,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667


In [60]:
cluster_result_consensus = hierarchical_clustering(
    distance_matrix_consensus,
    range_min=2,
    range_max=4,
    cluster_number=4,
    out_path=f"{result_out_path}/hierarchical_clustering_consensus.png",
)

chosen number of clusters: 4


In [61]:
cluster_result_consensus.head()

,gene,cluster
0,ACE,2
1,ACOT11,4
2,ACSF2,4
3,ADAMTS12,3
4,ADCY1,2


#### 5. Compare


In [62]:
from utils.clustering_helper import hierarchical_clustering_compare

hierarchical_clustering_compare(
    [weighted_sum_dataframe, simple_sum_dataframe, distance_matrix_consensus],
    ["Weighted Average", "Simple Average", "Consensus"],
    out_path=f"{result_out_path}/hierarchical_clustering_compare.png",
    range_min=2,
    range_max=4,
)

Best number of clusters for Weighted Average: 4
Best number of clusters for Simple Average: 2
Best number of clusters for Consensus: 4


In [63]:
dbeta_info = pd.read_csv(dbeta_file)

In [64]:
# column gene isin weighted_sum_dataframe
weighted_dbeta = dbeta_info[dbeta_info["gene"].isin(weighted_sum_dataframe.index)]
simple_dbeta = dbeta_info[dbeta_info["gene"].isin(simple_sum_dataframe.index)]
consensus_dbeta = dbeta_info[dbeta_info["gene"].isin(distance_matrix_consensus.index)]

In [65]:
weighted_dbeta.merge(cluster_result_weighted, on="gene").to_csv(
    f"{result_out_path}/{result_prefix}_weighted.csv", index=False
)
simple_dbeta.merge(cluster_result_simple, on="gene").to_csv(
    f"{result_out_path}/{result_prefix}_simple.csv", index=False
)
consensus_dbeta.merge(cluster_result_consensus, on="gene").to_csv(
    f"{result_out_path}/{result_prefix}_consensus.csv", index=False
)

### Sec. 6 SimpleModel Training


#### 1. Load Data


In [90]:
from utils.train_helper import read_selected_features_json
import pandas as pd
from utils.process_norm import *

In [93]:
dbeta_file = config["simple_model"]["hyper"]["dbeta_file"]
selected_feature_file = config["simple_model"]["hyper"]["selected_feature_file"]
train_out_path = config["simple_model"]["hyper"]["train_out_path"]
validate_out_path = config["simple_model"]["hyper"]["validate_out_path"]
df_file = config["simple_model"]["hyper"]["df_file"]
training_param_file = config["simple_model"]["hyper"]["training_param_file"]

In [94]:
selected_feature_file = read_selected_features_json(selected_feature_file)
dbeta_file = pd.read_csv(dbeta_file)

In [95]:
all_df = pd.read_csv(df_file)

In [98]:
import json
with open(f"../config/{TYPE}.json") as f:
    J = json.load(f)
data_source = "GDC_prostate_tissue"
# data_source = input("Enter the data source: ")

all_df = organize_dataset(all_df, J[data_source]["normal"], J[data_source]["tumor"], J[data_source]["sample_count"])

In [99]:
sp80, sp20 = split_dataset(all_df, 0.2, 42)

INFO complement_df feature: 373340
INFO complement_df sample (normal, tumor): (40, 401)
INFO ratio_df feature: 373340
INFO ratio_df sample (normal, tumor): (10, 101)


In [100]:
sp80

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,431,432,433,434,435,436,437,438,439,440
0,cg00000957,0.841180,0.853063,0.860960,0.862565,0.895689,0.831980,0.863311,0.845822,0.828084,...,0.885562,0.830317,0.852356,0.849449,0.869412,0.854675,0.892581,0.880069,0.857188,0.893333
1,cg00001349,0.708645,0.720905,0.860959,0.797306,0.891208,0.670794,0.715566,0.709133,0.814481,...,0.931344,0.724758,0.853462,0.878646,0.821124,0.833564,0.583747,0.876863,0.869314,0.785916
2,cg00001583,0.113957,0.279087,0.112048,0.046604,0.556086,0.110287,0.095149,0.072687,0.115331,...,0.731181,0.298437,0.357947,0.255105,0.493925,0.341537,0.132613,0.502308,0.739241,0.228809
3,cg00002028,0.030409,0.035949,0.022765,0.027793,0.016043,0.016709,0.028100,0.017500,0.039319,...,0.039711,0.027775,0.038226,0.039430,0.018812,0.027809,0.087653,0.037435,0.040489,0.031151
4,cg00002719,0.052727,0.045736,0.026899,0.042019,0.046145,0.051344,0.032892,0.030961,0.066875,...,0.821091,0.300829,0.447518,0.165109,0.408629,0.751978,0.368727,0.507314,0.769421,0.397939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.949989,0.948849,0.963573,0.951192,0.968733,0.947092,0.927369,0.953028,0.960975,...,0.974683,0.925931,0.935257,0.954601,0.941680,0.932389,0.936980,0.913569,0.853531,0.951475
373337,cg27657537,0.080608,0.277116,0.197913,0.101441,0.315259,0.070351,0.249224,0.202402,0.183052,...,0.736022,0.481165,0.407895,0.178566,0.468870,0.740228,0.389476,0.503911,0.488822,0.575861
373338,cg27662611,0.050093,0.042346,0.071625,0.065757,0.033946,0.018048,0.065646,0.024260,0.036035,...,0.077809,0.061533,0.044317,0.074580,0.028676,0.072972,0.048652,0.052870,0.057997,0.024181
373339,cg27665648,0.869885,0.879406,0.932071,0.931462,0.932957,0.878828,0.893683,0.875100,0.911748,...,0.902220,0.893676,0.873291,0.844760,0.865644,0.873046,0.902961,0.884115,0.899215,0.914803


In [101]:
sp20

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,101,102,103,104,105,106,107,108,109,110
0,cg00000957,0.851698,0.875340,0.873949,0.845864,0.845313,0.855674,0.852747,0.856824,0.892437,...,0.870900,0.878546,0.815294,0.832045,0.873123,0.867799,0.893290,0.874944,0.834560,0.849045
1,cg00001349,0.831998,0.818831,0.777109,0.773967,0.910121,0.827794,0.851243,0.779308,0.902253,...,0.842825,0.891249,0.671433,0.855471,0.771970,0.838953,0.860958,0.932942,0.883742,0.816816
2,cg00001583,0.727021,0.260109,0.258795,0.144976,0.059293,0.216979,0.070264,0.094708,0.383988,...,0.597724,0.337256,0.311713,0.558074,0.572805,0.611514,0.455211,0.660695,0.753027,0.671198
3,cg00002028,0.020635,0.019511,0.026367,0.027506,0.033701,0.040187,0.028661,0.020410,0.008625,...,0.023539,0.057423,0.046084,0.028183,0.032069,0.033905,0.032445,0.025413,0.040462,0.045968
4,cg00002719,0.764914,0.070651,0.027768,0.058274,0.053425,0.101438,0.128026,0.048165,0.522899,...,0.493793,0.702902,0.109746,0.408147,0.415052,0.406784,0.736912,0.671877,0.307906,0.693976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.955041,0.963189,0.962567,0.948103,0.959346,0.966859,0.949644,0.946614,0.970719,...,0.920759,0.976222,0.946938,0.936118,0.941034,0.956478,0.947787,0.980815,0.864471,0.890726
373337,cg27657537,0.589816,0.215809,0.296199,0.204987,0.132361,0.317090,0.106829,0.127561,0.576276,...,0.445997,0.622140,0.293959,0.389596,0.530595,0.532515,0.638457,0.685113,0.420863,0.478446
373338,cg27662611,0.023314,0.044780,0.038949,0.047586,0.045769,0.074387,0.040437,0.028859,0.047046,...,0.030432,0.066494,0.059676,0.031325,0.030693,0.032223,0.045322,0.094985,0.032344,0.046670
373339,cg27665648,0.910125,0.879289,0.894013,0.908067,0.935242,0.922943,0.860689,0.870091,0.920194,...,0.865312,0.903358,0.832897,0.915230,0.915200,0.898776,0.858458,0.908942,0.725667,0.858434


#### 2. Training


In [102]:
from utils.train_helper import set_parameters
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import json
from sklearn.ensemble import VotingClassifier


with open(training_param_file, "r") as f:
    training_param = json.load(f)

xgb_grid = set_parameters(XGBClassifier(random_state=42), training_param["XGBoost"])
rf_grid = set_parameters(RandomForestClassifier(random_state=42), training_param["RandomForest"])
svm_grid = set_parameters(SVC(random_state=42, probability=True), training_param["SVM"])
dt_grid = set_parameters(DecisionTreeClassifier(random_state=42), training_param["DecisionTree"])
voting = VotingClassifier(
    estimators=[("XGBoost", XGBClassifier(random_state=42)), ("RandomForest", RandomForestClassifier(random_state=42)), ("SVM", SVC(random_state=42, probability=True)), ("DecisionTree", DecisionTreeClassifier(random_state=42))
                ],
    voting="soft",
)

# comment out the model you don't want to use
models = {
    "XGBoost": {
        "is_grid_search": True,
        "model": xgb_grid,
    },
    "RandomForest": {
        "is_grid_search": True,
        "model": rf_grid,
    },
    "SVM": {
        "is_grid_search": True,
        "model": svm_grid,
    },
    "DecisionTree": {
        "is_grid_search": True,
        "model": dt_grid,
    },
    "Voting": {
        "is_grid_search": False,
        "model": voting,
    },
}

In [103]:
from utils.simple_model import SimpleModel

for model_name, gene_list in selected_feature_file.items():
    for model_name, model_config in models.items():
        model = SimpleModel(
            train_df=sp80,
            test_df=sp20,
            gene_list=gene_list,
            dbeta_info=dbeta_file,
        )
        model.setup_dbeta()
        model.setup_train_test()
        model.setup_combinations()
        model.train(
            model_name,
            model_config["model"],
            train_out_path,
            validate_out_path,
            model_config["is_grid_search"],
        )

INFO Training for combination: ('CDX2', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: XGBoost
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
INFO Training for combination: ('GCM2', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: XGBoost
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
INFO Training for combination: ('HAAO', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: XGBoost
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
INFO Training for combination: ('RNF152', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: XGBoost
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
INFO Training for combination: ('CDX2', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: RandomForest
Fitting 5 folds for each of 27 candidates, totalling 135 fits
INFO Training for combination: ('GCM2', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: RandomForest
Fitting 5 folds for each of 27 candidates, totalling 135 fits
INFO Training for combination: ('HAAO', 'ITIH5', '

#### 3. Visualization


In [104]:
model = input("Enter the model name: ")


##### Congratulation! You have finished the whole pipeline🎉. <br>

![title](cat.png)
